# Promethee II Ranking
This module shows an example of how to use the "Promethee II Ranking" module

In [1]:
%config Completer.use_jedi = False

## Definition of inputs and problem formalization

In [2]:
import pandas as pd
from modular_parts.M3_PrometheePreference import compute_preference_indices
from modular_parts.M8_PrometheeOutrankingFlows import calculate_promethee_outranking_flows
from modular_parts.M9_NetOutrankingFlow import calculate_net_outranking_flows
from modular_parts.M15_PrometheeIIRanking import calculate_promethee_ii_ranking
from core.enums import Direction, GeneralCriterion, FlowType


alternatives = [f"a{i}" for i in range(1, 8)]
profiles = [f"p{i}" for i in range(1, 4)]
criteria = [f"c{i}" for i in range(1, 4)]
criteria_directions = pd.Series([Direction.MAX, Direction.MIN, Direction.MAX], index=criteria)
criteria_weights = pd.Series([0.3, 0.2, 0.5], index=criteria)
generalised_criteria = pd.Series([GeneralCriterion.U_SHAPE,
                                  GeneralCriterion.V_SHAPE_INDIFFERENCE,
                                  GeneralCriterion.USUAL], index=criteria)
preference_thresholds = pd.Series([2, 10, None], index=criteria)
indifference_thresholds = pd.Series([1, 5, None], index=criteria)
standard_deviations = pd.Series([None, None, None], index=criteria) # None, because we do not use GeneralCriterion.GAUSSIAN

alternatives_performances = pd.DataFrame([[15, 83, 21],
                                          [10, 90, 15],
                                          [11, 75, 20],
                                          [18, 59, 20],
                                          [17, 60, 28],
                                          [22, 44, 15],
                                          [13, 62, 22]], index=alternatives, columns=criteria)

## Required data

### Required preferences

In [3]:
aggregated_alternatives_preferences, _ = compute_preference_indices(alternatives_performances,
                                                                    preference_thresholds,
                                                                    indifference_thresholds,
                                                                    standard_deviations,
                                                                    generalised_criteria,
                                                                    criteria_directions,
                                                                    criteria_weights)
aggregated_alternatives_preferences

,a1,a2,a3,a4,a5,a6,a7
a1,0.00,0.88,0.8,0.5,0.0,0.5,0.3
a2,0.00,0.00,0.0,0.0,0.0,0.0,0.0
a3,0.12,0.70,0.0,0.0,0.0,0.5,0.0
a4,0.50,1.00,0.5,0.0,0.0,0.5,0.3
a5,1.00,1.00,1.0,0.5,0.0,0.5,0.8
a6,0.50,0.50,0.5,0.5,0.5,0.0,0.5
a7,0.70,1.00,1.0,0.5,0.0,0.5,0.0


### Required flows

In [4]:
alternatives_flows = calculate_promethee_outranking_flows(aggregated_alternatives_preferences,
                                                          flow_type=FlowType.BASIC)
alternatives_net_flows = calculate_net_outranking_flows(alternatives_flows, True)  # temporary True
alternatives_net_flows

,positive,negative,net
a1,0.496667,0.470000,0.026667
a2,0.000000,0.846667,-0.846667
a3,0.220000,0.633333,-0.413333
a4,0.466667,0.333333,0.133333
a5,0.800000,0.083333,0.716667
a6,0.500000,0.416667,0.083333
a7,0.616667,0.316667,0.300000


## Promethee II Ranking

In [5]:
promethee_rank = calculate_promethee_ii_ranking(alternatives_net_flows)
promethee_rank

1    a5
2    a7
3    a4
4    a6
5    a1
6    a3
7    a2
Name: ranking, dtype: object